In [677]:
import pandas as pd


In [678]:
import ToolBox as tb

In [679]:
# Cargar el conjunto de datos de entrenamiento
train_data = pd.read_csv('./Data/train.csv')

# Mostrar los primeros registros del conjunto de datos
train_data.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [680]:
train_data['Ram'] = train_data['Ram'].str.replace('GB', '').astype(int)

In [681]:
train_data['Weight'] = train_data['Weight'].str.replace('kg', '').astype(float)

In [682]:
train_data['Memory'] = train_data['Memory'].str.replace('1.0TB', '1000GB', regex=False)
train_data['Memory'] = train_data['Memory'].str.replace('2.0TB', '2000GB', regex=False)
train_data['Memory'] = train_data['Memory'].str.replace('1.000TB', '1000GB', regex=False)
train_data['Memory'] = train_data['Memory'].str.replace('TB', '000GB', regex=False)

In [683]:
def extraer_memoria(memory):
    ssd, hdd, flash, hybrid = 0, 0, 0, 0
    for x in memory.split('+'):
        x=x.strip()
        if 'SSD' in x:
            ssd = int(x.replace('SSD', '').replace('GB', '').strip())
        elif 'HDD' in x:
           hdd = int(x.replace('HDD', '').replace('GB', '').strip())
        elif 'Flash' in x:
            flash = int(x.replace('GB', '').replace('Flash Storage','').strip())
        elif 'Hybrid' in x:
            hybrid = int(x.replace('GB', '').replace('Hybrid','').strip())
            
            
    return pd.Series([ssd, hdd, flash, hybrid])
    

In [684]:
train_data[['SSD', 'HDD', 'Flash', 'Hybrid']] = train_data['Memory'].apply(extraer_memoria)

In [685]:
train_data['Gpu']=train_data['Gpu'].str.strip()

In [686]:
def extraer_gpu(gpu):
    if 'Radeon' in gpu or 'Firepro' in gpu:
        return 'Radeon' if 'Radeon' in gpu else 'Firepro'
    elif 'AMD' in gpu:
        return 'Radeon'
    elif 'UHD' in gpu or 'Iris' in gpu:
        return 'UHD' if 'UHD' in gpu else 'Iris'
    elif 'Intel' in gpu:
        return 'HD'
    elif 'Quadro' in gpu:
        return 'Quadro'
    elif 'Nvidia' in gpu:
        return 'GeForce'

train_data['Gpu_Type'] = train_data['Gpu'].apply(extraer_gpu)

print(train_data[['Gpu', 'Gpu_Type']].head())


                       Gpu Gpu_Type
0       AMD Radeon R7 M445   Radeon
1    Intel HD Graphics 505       HD
2    Intel HD Graphics 520       HD
3  Nvidia GeForce GTX 1070  GeForce
4    Intel HD Graphics 520       HD


In [687]:
import re

In [688]:
def extraer_cpu(cpu):
    tipo = None
    tipo2 = None
    if 'AMD' in cpu:
        tipo = 'AMD'
        T = cpu.split()[1]
        if 'A' in T:
            tipo2 = 'A-Series'
        elif 'FX' in T:
            tipo2 = 'FX'
        elif 'E' in T:
            tipo2 = 'E-Series'
        elif 'Ryzen' in T:
            tipo2 = 'Ryzen'
    elif 'Intel' in cpu:
        if 'Celeron' in cpu:
            tipo = 'Celeron'
            if 'Dual' in cpu:
                tipo2 = 'Dual'
            else:
                tipo2 = 'Quad'
        elif 'Pentium' in cpu:
            tipo = 'Pentium'
            if 'Dual' in cpu:
                tipo2 = 'Dual'
            else:
                tipo2 = 'Quad'
        elif 'Atom' in cpu:
            tipo = 'Atom'
            T = cpu.split()[2]
            if 'x' in T:
                tipo2 = 'X-Series'
            else:
                tipo2 = 'Z-Series'
        elif 'Xeon' in cpu:
            tipo = 'Xeon'
            tipo2='E-Series'
        else:
            tipo = cpu.split()[2]
            if tipo == 'M':
                tipo2 = 'M'
            else:
                tipo2 = re.search(r'\d+([A-Za-z])', cpu).group(1) if re.search(r'\d+([A-Za-z])', cpu) else 'Unknown'
    return pd.Series([tipo, tipo2])

In [689]:
train_data[['Cpu_Tipo', 'Cpu_Tipo2']]=train_data['Cpu'].apply(extraer_cpu)

In [690]:
def is_ips(screen_resolution):
    return True if 'IPS' in screen_resolution else False 

In [691]:
def is_touchscreen(screen_resolution):
    return True if 'Touchscreen' in screen_resolution else False  

In [692]:
train_data['Is_IPS'] = train_data['ScreenResolution'].apply(is_ips)

In [693]:
train_data['IS_TouchScreen'] = train_data['ScreenResolution'].apply(is_touchscreen)

In [694]:
def get_num_pixels(screen_resolution):
    # Buscar el patrón de resolución
    match = re.search(r'(\d+)x(\d+)', screen_resolution)
    if match:
        width = int(match.group(1))
        height = int(match.group(2))
        return width * height
    return None

In [695]:
train_data['num_pixels']  = train_data['ScreenResolution'].apply(get_num_pixels)

In [696]:
train_data[['ScreenResolution','Is_IPS','IS_TouchScreen','num_pixels']]

,ScreenResolution,Is_IPS,IS_TouchScreen,num_pixels
0,1366x768,False,False,1049088
1,1366x768,False,False,1049088
2,Full HD 1920x1080,False,False,2073600
3,IPS Panel 4K Ultra HD 3840x2160,True,False,8294400
4,Full HD / Touchscreen 1920x1080,False,True,2073600
...,...,...,...,...
907,IPS Panel Touchscreen 1366x768,True,True,1049088
908,Full HD 1920x1080,False,False,2073600
909,1366x768,False,False,1049088
910,Full HD 1920x1080,False,False,2073600


In [697]:
gpu_dummies = pd.get_dummies(train_data['Gpu_Type'], prefix='Gpu_Type')
train_data = pd.concat([train_data, gpu_dummies], axis=1)
train_data = train_data.drop(columns=['Gpu', 'Gpu_Type'])

In [698]:
cpu_tipo_dummies = pd.get_dummies(train_data['Cpu_Tipo'], prefix='Cpu_Tipo')
cpu_tipo2_dummies = pd.get_dummies(train_data['Cpu_Tipo2'], prefix='Cpu_Tipo2')
train_data = pd.concat([train_data, cpu_tipo_dummies, cpu_tipo2_dummies], axis=1)
train_data = train_data.drop(columns=['Cpu', 'Cpu_Tipo', 'Cpu_Tipo2'])

In [699]:
train_data['OpSys'] = train_data['OpSys'].str.replace('10 S', '10').astype(str)

In [700]:
OpSys_dummies = pd.get_dummies(train_data['OpSys'], prefix='OpSys')
train_data = pd.concat([train_data, OpSys_dummies], axis=1)
train_data = train_data.drop(columns=['OpSys'])

In [701]:
Company_dummies = pd.get_dummies(train_data['Company'], prefix='Company')
train_data = pd.concat([train_data, Company_dummies], axis=1)
train_data = train_data.drop(columns=['Company'])

In [702]:
TypeName_dummies = pd.get_dummies(train_data['TypeName'], prefix='TypeName')
train_data = pd.concat([train_data, TypeName_dummies], axis=1)
train_data = train_data.drop(columns=['TypeName'])

In [703]:
features = ['Inches','Ram', 'Weight', 'SSD', 'HDD', 'Flash', 'Hybrid', 'Is_IPS', 'num_pixels', 'IS_TouchScreen']
features += [col for col in train_data.columns if col.startswith('Gpu_Type_')]
features += [col for col in train_data.columns if col.startswith('Cpu_Tipo_')]
features += [col for col in train_data.columns if col.startswith('OpSys')]
features += [col for col in train_data.columns if col.startswith('Company')]
features += [col for col in train_data.columns if col.startswith('TypeName')]

# Test

In [704]:
# Cargar el conjunto de datos de prueba
test_data = pd.read_csv('./Data/test.csv')

In [705]:
test_data.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,181,1098,HP,Spectre x360,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.3kg
1,708,330,Acer,Aspire 5,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8GB,256GB SSD,AMD Radeon RX 540,Windows 10,2.2kg
2,862,1260,Acer,Aspire ES1-572,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,500GB HDD,Intel HD Graphics 520,Linux,2.4kg
3,1064,1137,HP,EliteBook 1040,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.43kg
4,702,1015,HP,ENVY -,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.34kg


In [706]:
test_data['Ram'] = test_data['Ram'].str.replace('GB', '').astype(int)    

In [707]:
test_data['Weight'] = test_data['Weight'].str.replace('kg', '').astype(float)

In [708]:
test_data['Memory'] = test_data['Memory'].str.replace('1.0TB', '1000GB', regex=False)
test_data['Memory'] = test_data['Memory'].str.replace('2.0TB', '2000GB', regex=False)
test_data['Memory'] = test_data['Memory'].str.replace('1.000TB', '1000GB', regex=False)
test_data['Memory'] = test_data['Memory'].str.replace('TB', '000GB', regex=False)

In [709]:
test_data[['SSD', 'HDD', 'Flash', 'Hybrid']] = test_data['Memory'].apply(extraer_memoria)
test_data['Gpu'] = test_data['Gpu'].str.strip()

In [710]:
test_data['Gpu_Type'] = test_data['Gpu'].apply(extraer_gpu)

In [711]:
test_data[['Cpu_Tipo', 'Cpu_Tipo2']] = test_data['Cpu'].apply(extraer_cpu)

In [712]:
test_data['Is_IPS'] = test_data['ScreenResolution'].apply(is_ips)
test_data['IS_TouchScreen'] = test_data['ScreenResolution'].apply(is_touchscreen)

In [713]:
test_data['num_pixels'] = test_data['ScreenResolution'].apply(get_num_pixels)

gpu_dummies_test = pd.get_dummies(test_data['Gpu_Type'], prefix='Gpu_Type')
test_data = pd.concat([test_data, gpu_dummies_test], axis=1)
test_data = test_data.drop(columns=['Gpu', 'Gpu_Type'])

cpu_tipo_dummies_test = pd.get_dummies(test_data['Cpu_Tipo'], prefix='Cpu_Tipo')
cpu_tipo2_dummies_test = pd.get_dummies(test_data['Cpu_Tipo2'], prefix='Cpu_Tipo2')
test_data = pd.concat([test_data, cpu_tipo_dummies_test, cpu_tipo2_dummies_test], axis=1)
test_data = test_data.drop(columns=['Cpu', 'Cpu_Tipo', 'Cpu_Tipo2'])

test_data['OpSys'] = test_data['OpSys'].str.replace('10 S', '10').astype(str)

OpSys_dummies_test = pd.get_dummies(test_data['OpSys'], prefix='OpSys')
test_data = pd.concat([test_data, OpSys_dummies_test], axis=1)
test_data = test_data.drop(columns=['OpSys'])

Company_dummies_test = pd.get_dummies(test_data['Company'], prefix='Company')
test_data = pd.concat([test_data, Company_dummies_test], axis=1)
test_data = test_data.drop(columns=['Company'])

TypeName_dummies_test = pd.get_dummies(test_data['TypeName'], prefix='TypeName')
test_data = pd.concat([test_data, TypeName_dummies_test], axis=1)
test_data = test_data.drop(columns=['TypeName'])

In [714]:
features_test = ['Inches', 'Ram', 'Weight', 'SSD', 'HDD', 'Flash', 'Hybrid', 'Is_IPS', 'num_pixels', 'IS_TouchScreen']
features_test += [col for col in test_data.columns if col.startswith('Gpu_Type_')]
features_test += [col for col in test_data.columns if col.startswith('Cpu_Tipo_')]
features_test += [col for col in test_data.columns if col.startswith('OpSys')]
features_test += [col for col in test_data.columns if col.startswith('Company')]
features_test += [col for col in test_data.columns if col.startswith('TypeName')]

In [715]:
test_data[num_features] = scaler.transform(test_data[num_features])

# Selección de modelo

In [716]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [717]:
features_train=features

In [718]:
features=[x for x in features_test if x in features_train]

In [720]:
from sklearn.linear_model import Ridge

In [721]:
# Características y objetivo
X = train_data[features]
y = train_data['Price_euros']

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características numéricas
scaler = StandardScaler()
num_features = ['Inches', 'Ram', 'Weight', 'SSD', 'HDD', 'Flash', 'Hybrid', 'num_pixels']
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_val[num_features] = scaler.transform(X_val[num_features])

# Entrenar el modelo con Ridge
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred = model.predict(X_val)

# Evaluar el modelo utilizando MAE
mae = mean_absolute_error(y_val, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 212.83192163793296


In [723]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Definir el modelo
rf = RandomForestRegressor(random_state=42)

# Definir los parámetros para GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear el GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Entrenar el modelo
grid_search.fit(X_train, y_train)

# Mejor estimador
best_rf = grid_search.best_estimator_

# Predecir en el conjunto de validación
y_pred = best_rf.predict(X_val)

# Evaluar el modelo utilizando MAE
mae = mean_absolute_error(y_val, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Best Parameters: {grid_search.best_params_}")

Fitting 3 folds for each of 4320 candidates, totalling 12960 fits


c:\Users\aritz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
6480 fits failed out of a total of 12960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2343 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\aritz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\aritz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\aritz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\

Mean Absolute Error (MAE): 177.57922649744344
Best Parameters: {'bootstrap': False, 'max_depth': 15, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [724]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

In [725]:
# Definir el modelo y los parámetros para GridSearchCV
lasso = Lasso()
param_grid_lasso = {
    'alpha': [0.01, 0.1, 1, 10, 100]
}

# Crear y ajustar GridSearchCV
grid_search_lasso = GridSearchCV(estimator=lasso, param_grid=param_grid_lasso, cv=5, n_jobs=-1, verbose=2)
grid_search_lasso.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred_lasso = grid_search_lasso.best_estimator_.predict(X_val)

# Evaluar el modelo utilizando MAE
mae_lasso = mean_absolute_error(y_val, y_pred_lasso)
print(f"Mean Absolute Error (MAE) para Lasso: {mae_lasso}")
print(f"Mejores parámetros para Lasso: {grid_search_lasso.best_params_}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Mean Absolute Error (MAE) para Lasso: 211.24216233561847
Mejores parámetros para Lasso: {'alpha': 1}


In [ ]:
# Definir el modelo y los parámetros para GridSearchCV
xgbr = XGBRegressor(objective='reg:squarederror', random_state=42)
param_grid_xgbr = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.05],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5]
}
# Crear y ajustar GridSearchCV
grid_search_xgbr = GridSearchCV(estimator=xgbr, param_grid=param_grid_xgbr, cv=5, n_jobs=-1, verbose=2)
grid_search_xgbr.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred_xgbr = grid_search_xgbr.best_estimator_.predict(X_val)

# Evaluar el modelo utilizando MAE
mae_xgbr = mean_absolute_error(y_val, y_pred_xgbr)
print(f"Mean Absolute Error (MAE) para XGBRegressor: {mae_xgbr}")
print(f"Mejores parámetros para XGBRegressor: {grid_search_xgbr.best_params_}")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Definir el modelo y los parámetros para GridSearchCV
gbr = GradientBoostingRegressor(random_state=42)
param_grid_gbr = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.05],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

# Crear y ajustar GridSearchCV
grid_search_gbr = GridSearchCV(estimator=gbr, param_grid=param_grid_gbr, cv=5, n_jobs=-1, verbose=2)
grid_search_gbr.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred_gbr = grid_search_gbr.best_estimator_.predict(X_val)

# Evaluar el modelo utilizando MAE
mae_gbr = mean_absolute_error(y_val, y_pred_gbr)
print(f"Mean Absolute Error (MAE) para GradientBoostingRegressor: {mae_gbr}")
print(f"Mejores parámetros para GradientBoostingRegressor: {grid_search_gbr.best_params_}")

# Los mejores osn el de GradientBoosing y XGBRegressor

In [ ]:
y_test_pred =  grid_search_xgbr.best_estimator_.predict(test_data[features])

NameError: name 'grid_search_xgbr' is not defined

In [ ]:
predictions = pd.DataFrame({
    'id': test_data.id,
    'Price_euros': y_test_pred
})


# Comprobacion

In [ ]:
sample = pd.read_csv("Data/sample_submission.csv") # Esta es mi ruta del archivo, la vuestra puede ser otra

In [ ]:
sample.head()

,id,Price_euros
0,1014,752.0
1,845,499.0
2,1151,1747.0
3,1265,245.0
4,573,1179.0


In [ ]:
sample.shape

(391, 2)

In [ ]:
predictions.head()

,id,Price_euros
0,181,1614.485743
1,708,535.605934
2,862,373.651739
3,1064,1441.145859
4,702,1126.874383


In [ ]:
predictions.shape

(391, 2)

In [ ]:
import urllib
from PIL import Image

In [ ]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                predictions.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [ ]:
predictions

,id,Price_euros
0,181,1614.485743
1,708,535.605934
2,862,373.651739
3,1064,1441.145859
4,702,1126.874383
...,...,...
386,1281,950.959765
387,524,2682.596314
388,1015,527.785845
389,1236,493.314082


In [ ]:
chequeator(predictions) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

You're ready to submit!
